# Load Data

In [1]:
import pandas as pd

In [2]:
data_1 = pd.read_csv(r'D:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\notebooks\articulos_0-132.csv')

In [3]:
data_1.head(3)

,articulo_id,articulo_name,detalla,contenido,conteo_palabras,rango_palabras
0,4,Artículo 4,Constitución,Un edificio o conjunto se somete al régimen de...,36,0-132
1,6,Artículo 6,Documentación anexa,Con la escritura pública de constitución o de ...,109,0-132
2,10,Artículo 10,Procedimiento,La propiedad horizontal se extingue total o pa...,52,0-132


In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   articulo_id      42 non-null     int64 
 1   articulo_name    42 non-null     object
 2   detalla          42 non-null     object
 3   contenido        41 non-null     object
 4   conteo_palabras  42 non-null     int64 
 5   rango_palabras   42 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.1+ KB


In [4]:
df = data_1.copy()

In [7]:
from dotenv import load_dotenv
import os

In [8]:
load_dotenv()

True

In [9]:
H_FACES_KEY = os.getenv('HFACES')

# LOAD MODEL

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
from huggingface_hub import login

In [13]:
# Autenticar con el token
login(H_FACES_KEY)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\USUARIO\.cache\huggingface\token
Login successful


In [14]:
# Descargar el tokenizer y el modelo desde Hugging Face
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USUARIO\.cache\huggingface\hub\models--google--gemma-2-2b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [15]:
# Guardar el modelo localmente
model.save_pretrained("./model/gemma")
tokenizer.save_pretrained("./model/gemma")

('./model/gemma\\tokenizer_config.json',
 './model/gemma\\special_tokens_map.json',
 './model/gemma\\tokenizer.json')

In [15]:
ruta_modelo = r'D:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\notebooks\model\gemma'

In [17]:
# Cargar el tokenizer y el modelo (ajusta la ruta al modelo local)
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)
model = AutoModelForCausalLM.from_pretrained(ruta_modelo)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
# Ejemplo de uso
def hacer_pregunta(pregunta):
    inputs = tokenizer(pregunta, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=100)
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta

In [19]:
pregunta = "¿Qué es la inteligencia artificial?"
print(hacer_pregunta(pregunta))

KeyboardInterrupt: 

In [5]:
import torch

In [6]:
# Verifica si hay GPU disponible
if torch.cuda.is_available():
    print("GPU está disponible")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU no está disponible, usando CPU")

GPU no está disponible, usando CPU


## Api

In [15]:
from huggingface_hub import InferenceClient

In [16]:
# Función para generar preguntas con ChatGPT
def generar_preguntas(contenido_articulo, num_preguntas):
    
    
    # Crear el prompt con el contenido del artículo
    prompt = f"Genera {num_preguntas} preguntas basadas en el siguiente contenido:\n{contenido_articulo}\n"
    
    # Realizar la solicitud al cliente
    response = client.chat_completion(
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,  # Ajusta según el tamaño que esperes
        stream=False  # Si no necesitas respuesta en streaming
    )
    
    # Extraer la respuesta del modelo
    preguntas = response['choices'][0]['message']['content'].split('\n')
    
    # Limpiar las preguntas y eliminar cualquier línea vacía
    preguntas = [pregunta.strip() for pregunta in preguntas if pregunta]

    return preguntas


In [1]:
# DataFrame vacío para almacenar las preguntas generadas
df_preguntas = pd.DataFrame(columns=['pregunta_id', 'pregunta', 'articulo_id', 'embeding'])

NameError: name 'pd' is not defined

In [21]:
# Generar preguntas para cada artículo
pregunta_id = 0
for _, row in df.iterrows():
    articulo_id = row['articulo_id']
    contenido_articulo = row['contenido']
    
    preguntas = generar_preguntas(contenido_articulo, num_preguntas=1)
    
    for pregunta in preguntas:
        embeding = ''
        df_preguntas = df_preguntas.append({
            'pregunta_id': pregunta_id,
            'pregunta': pregunta,            
            'articulo_id': articulo_id,
            'embeding': embeding,
        }, ignore_index=True)
        
        pregunta_id += 1

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-2-9b-it/v1/chat/completions (Request ID: ZzEkhHApJcJ0o-2kgVOJ1)

Model too busy, unable to get response in less than 60 second(s)

In [ ]:
# Generar preguntas para cada artículo
preguntas_list = df.apply(lambda row: [
    {
        'pregunta_id': i,
        'pregunta': pregunta,
        'articulo_id': row['articulo_id'],
        'embeding': ''
    }
    for i, pregunta in enumerate(generar_preguntas(row['contenido'], num_preguntas=1))
], axis=1).explode().tolist()

# Crear el DataFrame de preguntas
df_preguntas = pd.DataFrame({
    'pregunta_id': [i for i, _ in enumerate(preguntas_list)],
    'pregunta': [p['pregunta'] for p in preguntas_list],
    'articulo_id': [p['articulo_id'] for p in preguntas_list],
    'embeding': [p['embeding'] for p in preguntas_list]
})

In [ ]:
df_preguntas.sample(5)

### Generar preguntas

In [ ]:
pregunta_id_counter = 0
# Generar preguntas para cada artículo
preguntas_list = []
for _, row in df.iterrows():
    # Generar preguntas para el contenido del artículo
    preguntas = generar_preguntas(row['contenido'], max_q=3)
    
    # Agregar cada pregunta a la lista
    for pregunta in preguntas:
        preguntas_list.append({
            'pregunta_id': f'{pregunta_id_counter}_{row["articulo_id"]}',
            'pregunta': pregunta,
            'articulo_id': row['articulo_id'],
            'embeding': ''  # Si planeas agregar embeddings después
        })
        # Incrementar el contador de pregunta_id
        pregunta_id_counter += 1

# Crear el DataFrame de preguntas
df_preguntas = pd.DataFrame.from_records(preguntas_list)